#  UR5 Manipulator Sensor Data Exploration – Notebook 01

**Objective:**  
This notebook performs **initial data exploration and structural validation** for UR5 robotic manipulator controller logs.  

**Data Sources:**
- `ur5testresult_header.xlsx`: Defines column names, types, and meanings.
- `calculated_deviation_of_actual_position_to_nominal_position.xlsx`: Contains deviation summaries (validation reference).
- Multiple `.csv` sensor data files: Controller-level logs at 125Hz across various payloads and test conditions.


##  Step 1: Import Dependencies and Define Paths
We start by loading essential libraries and defining directory paths for raw and processed data.


In [1]:
# --- Step 1: Imports & Paths ---
import pandas as pd
import numpy as np
import os
import glob

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

# Paths
header_path = "../data/raw/header/ur5testresult_header.xlsx"
sensor_path = "../data/raw/sensor_data/*.csv"
summary_path = "../data/raw/summary/calculated_deviation_of_actual_position_to_nominal_position.xlsx"

##  Step 2: Load and Inspect Header Metadata
This file contains descriptions and definitions for each column in the sensor dataset.  
It’s used to validate data consistency and understand what each signal represents.


In [2]:
# --- Step 2: Header Metadata ---
header_meta = pd.read_excel(header_path)
print(f"Header metadata shape: {header_meta.shape}")
header_meta.head()

Header metadata shape: (0, 73)


,ROBOT_TIME,ROBOT_TARGET_JOINT_POSITIONS (J1),ROBOT_TARGET_JOINT_POSITIONS (J2),ROBOT_TARGET_JOINT_POSITIONS (J3),ROBOT_TARGET_JOINT_POSITIONS (J4),ROBOT_TARGET_JOINT_POSITIONS (J5),ROBOT_TARGET_JOINT_POSITIONS (J6),ROBOT_ACTUAL_JOINT_POSITIONS (J1),ROBOT_ACTUAL_JOINT_POSITIONS (J2),ROBOT_ACTUAL_JOINT_POSITIONS (J3),ROBOT_ACTUAL_JOINT_POSITIONS (J4),ROBOT_ACTUAL_JOINT_POSITIONS (J5),ROBOT_ACTUAL_JOINT_POSITIONS (J6),ROBOT_TARGET_JOINT_VELOCITIES (J1),ROBOT_TARGET_JOINT_VELOCITIES (J2),ROBOT_TARGET_JOINT_VELOCITIES (J3),ROBOT_TARGET_JOINT_VELOCITIES (J4),ROBOT_TARGET_JOINT_VELOCITIES (J5),ROBOT_TARGET_JOINT_VELOCITIES (J6),ROBOT_ACTUAL_JOINT_VELOCITIES (J1),ROBOT_ACTUAL_JOINT_VELOCITIES (J2),ROBOT_ACTUAL_JOINT_VELOCITIES (J3),ROBOT_ACTUAL_JOINT_VELOCITIES (J4),ROBOT_ACTUAL_JOINT_VELOCITIES (J5),ROBOT_ACTUAL_JOINT_VELOCITIES (J6),ROBOT_TARGET_JOITN_CURRENT (J1),ROBOT_TARGET_JOITN_CURRENT (J2),ROBOT_TARGET_JOITN_CURRENT (J3),ROBOT_TARGET_JOITN_CURRENT (J4),ROBOT_TARGET_JOITN_CURRENT (J5),ROBOT_TARGET_JOITN_CURRENT (J6),ROBOT_ACTUAL_JOINT_CURRENT (J1),ROBOT_ACTUAL_JOINT_CURRENT (J2),ROBOT_ACTUAL_JOINT_CURRENT (J3),ROBOT_ACTUAL_JOINT_CURRENT (J4),ROBOT_ACTUAL_JOINT_CURRENT (J5),ROBOT_ACTUAL_JOINT_CURRENT (J6),ROBOT_TARGET_JOINT_ACCELERATIONS (J1),ROBOT_TARGET_JOINT_ACCELERATIONS (J2),ROBOT_TARGET_JOINT_ACCELERATIONS (J3),ROBOT_TARGET_JOINT_ACCELERATIONS (J4),ROBOT_TARGET_JOINT_ACCELERATIONS (J5),ROBOT_TARGET_JOINT_ACCELERATIONS (J6),ROBOT_TARGET_JOINT_TORQUES (J1),ROBOT_TARGET_JOINT_TORQUES (J2),ROBOT_TARGET_JOINT_TORQUES (J3),ROBOT_TARGET_JOINT_TORQUES (J4),ROBOT_TARGET_JOINT_TORQUES (J5),ROBOT_TARGET_JOINT_TORQUES (J6),ROBOT_JOINT_CONTROL_CURRENT (J1),ROBOT_JOINT_CONTROL_CURRENT (J2),ROBOT_JOINT_CONTROL_CURRENT (J3),ROBOT_JOINT_CONTROL_CURRENT (J4),ROBOT_JOINT_CONTROL_CURRENT (J5),ROBOT_JOINT_CONTROL_CURRENT (J6),ROBOT_CARTESIAN_COORD_TOOL (x),ROBOT_CARTESIAN_COORD_TOOL (y),ROBOT_CARTESIAN_COORD_TOOL (z),ROBOT_CARTESIAN_COORD_TOOL (rx),ROBOT_CARTESIAN_COORD_TOOL (ry),ROBOT_CARTESIAN_COORD_TOOL (rz),ROBOT_TCP_FORCE (x),ROBOT_TCP_FORCE (y),ROBOT_TCP_FORCE (z),ROBOT_TCP_FORCE (rx),ROBOT_TCP_FORCE (ry),ROBOT_TCP_FORCE (rz),ROBOT_JOINT_TEMP (J1),ROBOT_JOINT_TEMP (J2),ROBOT_JOINT_TEMP (J3),ROBOT_JOINT_TEMP (J4),ROBOT_JOINT_TEMP (J5),ROBOT_JOINT_TEMP (J6)


In [3]:
print("\n--- Full List of Headers from the Excel File ---")
# Print all column names directly
print(header_meta.columns.tolist())


--- Full List of Headers from the Excel File ---
['ROBOT_TIME', 'ROBOT_TARGET_JOINT_POSITIONS (J1)', 'ROBOT_TARGET_JOINT_POSITIONS (J2)', 'ROBOT_TARGET_JOINT_POSITIONS (J3)', 'ROBOT_TARGET_JOINT_POSITIONS (J4)', 'ROBOT_TARGET_JOINT_POSITIONS (J5)', 'ROBOT_TARGET_JOINT_POSITIONS (J6)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J1)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J2)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J3)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J4)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J5)', 'ROBOT_ACTUAL_JOINT_POSITIONS (J6)', 'ROBOT_TARGET_JOINT_VELOCITIES (J1)', 'ROBOT_TARGET_JOINT_VELOCITIES (J2)', 'ROBOT_TARGET_JOINT_VELOCITIES (J3)', 'ROBOT_TARGET_JOINT_VELOCITIES (J4)', 'ROBOT_TARGET_JOINT_VELOCITIES (J5)', 'ROBOT_TARGET_JOINT_VELOCITIES (J6)', 'ROBOT_ACTUAL_JOINT_VELOCITIES (J1)', 'ROBOT_ACTUAL_JOINT_VELOCITIES (J2)', 'ROBOT_ACTUAL_JOINT_VELOCITIES (J3)', 'ROBOT_ACTUAL_JOINT_VELOCITIES (J4)', 'ROBOT_ACTUAL_JOINT_VELOCITIES (J5)', 'ROBOT_ACTUAL_JOINT_VELOCITIES (J6)', 'ROBOT_TARGET_JOITN_CURRENT (J1)', 

##  Step 3: Detect and Inspect Raw Sensor Data Files
All CSV files are located in `../data/raw/sensor_data/`.  
Each file likely corresponds to a test run or a specific measurement scenario.
We’ll read and combine them into a single DataFrame for analysis.
Initial inspection helps understand data structure.

In [4]:
# --- Step 3: Sensor Data ---
sensor_files = sorted(glob.glob(sensor_path))

print("✔ Total sensor CSV files detected:", len(sensor_files))
print("\nList of Detected Sensor Files:")
for file in sensor_files:
    print(file)

✔ Total sensor CSV files detected: 18

List of Detected Sensor Files:
../data/raw/sensor_data/ur5testresult_coldstart_fullspeed_payload4.5lb_1.csv
../data/raw/sensor_data/ur5testresult_coldstart_fullspeed_payload4.5lb_2.csv
../data/raw/sensor_data/ur5testresult_coldstart_fullspeed_payload4.5lb_3.csv
../data/raw/sensor_data/ur5testresult_coldstart_halfspeed_payload4.5lb_1.csv
../data/raw/sensor_data/ur5testresult_coldstart_halfspeed_payload4.5lb_2.csv
../data/raw/sensor_data/ur5testresult_coldstart_halfspeed_payload4.5lb_3.csv
../data/raw/sensor_data/ur5testresult_fullspeed_payload1.6lb_1_1.csv
../data/raw/sensor_data/ur5testresult_fullspeed_payload1.6lb_2.csv
../data/raw/sensor_data/ur5testresult_fullspeed_payload1.6lb_3.csv
../data/raw/sensor_data/ur5testresult_fullspeed_payload4.5lb_1.csv
../data/raw/sensor_data/ur5testresult_fullspeed_payload4.5lb_2.csv
../data/raw/sensor_data/ur5testresult_fullspeed_payload4.5lb_3.csv
../data/raw/sensor_data/ur5testresult_halfspeed_payload1.6lb_1.c

In [5]:
# Read all sensor CSVs
sensor_dfs = []
for file in sensor_files:
    df = pd.read_csv(file)
    df["source_file"] = file.split("/")[-1]  # Keep track of file origin
    sensor_dfs.append(df)

# Combine all into one DataFrame
sensor_data = pd.concat(sensor_dfs, ignore_index=True)

print(f"Loaded {len(sensor_files)} sensor files")
print(f"Combined dataset shape: {sensor_data.shape}")
display(sensor_data.head())


Loaded 18 sensor files
Combined dataset shape: (153640, 806)


,([747.2479999999999],[-26.880068716264294,-79.91160892471794,57.09539150546534,-157.7717644023623,-105.0096132254245,-44.72477900700451],[-26.876620441547427,-79.91090832767539,57.096775121424834,-157.77315241285316,-105.0075641686735,-44.72546202592151],[0.0,0.0,0.0.1,0.0.2,0.0.3,0.0],[0.0.1,0.0.4,-0.0,0.0.5,0.0.6,0.0].1,[-2.2329763972194327e-18,-2.2138144127730115,-1.5893478394401743,-0.1629906069639267,0.00045060505107219365,0.0].2,[0.2398737221956253,-3.4344537258148193,-1.869670033454895,-0.3095833659172058,-0.20893064141273499,-0.10675287991762161],[0.0.2,0.0.7,0.0.8,0.0.9,0.0.10,0.0].3,[-2.6149525002185862e-17,-25.67417061620775,-18.441131024188945,-1.3760677172009348,0.003847821308427089,0.0].4,[0.22866468131542206,-3.4344537258148193.1,-1.8629446029663086,-0.3095833659172058.1,-0.19368022680282593,-0.11742816865444183],[-0.6377188174812196,0.2775360293865634,0.7569954955608781,-1.0750343619762845,-1.1303152748680048,0.045501548143443295],[-25.23140494271829,17.43970690181103,6.516587900455988,-1.0051605571420474,0.39324298820067416,0.9694435483534404],[25.209991455078125,26.71473503112793,26.71240997314453,29.80539321899414,28.6905517578125,29.992847442626953]),source_file,([838.496],-157.77176440236232,[-26.875937422630425,57.097485461098515,-157.77246939393615,-105.0102962443415,[-1.3009621579908202e-17,-2.213814412773011,-1.589347839440174,-0.16299060696392653,0.0004506050510722241,[0.33402976393699646,-3.4254865646362305,-2.0759165287017822,-0.3233087360858917,-0.17690478265285492,-0.14792899787425995],[-1.5235065860812838e-16,-25.674170616207746,-18.44113102418894,-1.3760677172009337,0.0038478213084273487,[0.33402976393699646.1,-3.4187610149383545,-2.093851089477539,-0.3187336027622223,-0.1830049455165863,-0.14487890899181366],[-0.6377173434435497,0.2775399064172585,0.7569847328371765,-1.0750672779617345,-1.130315967981058,0.04546842261775975],[-15.612004252178144,17.423084588992484,-4.151516821319487,-0.11602661663899572,1.9500671604014905,2.392210649582704],[25.459991455078125,26.99590301513672,26.90947914123535,30.19999122619629,29.218387603759766,30.481191635131836]),([918.52],[-27.962264024897266,-75.21065150153109,57.096231983991736,-161.18834855313767,-105.00895644940157,-44.724120726946786],[-27.963330859253134,-75.20974376346898,57.09472606467383,-161.1899682055304,-105.0089302065075,[-0.0,-0.0.1,[1.5192480626313783e-17,-2.54899473026629,-1.6393107979211894,-0.168755346377581,-0.0005343031817187057,[-0.17261941730976105,-2.833648443222046,-2.1364455223083496,-0.3690599799156189,-0.1891051083803177,-0.11895321309566498],[1.7791327865252693e-16,-29.561342281938277,-19.020848969399132,-1.4247372200205741,-0.0045625391079964625,[-0.17486122250556946,-2.824681282043457,-2.107301950454712,-0.37058499455451965,-0.1891051083803177.1,[-0.6742423471852145,0.3116971903029931,0.7135476978471418,-1.0971811187667286,-1.138870065087788,0.026005268468108286],[10.325820773057233,-3.8103216874590706,-15.796534930339947,-0.21334015519758312,1.4856598476590153,2.9159562810413076],[25.679704666137695,27.249340057373047,27.087356567382812,30.579988479614258,29.611787796020508,30.919404983520508]),([306.168],[-26.88279493255712,-79.91298470518308,57.093360026839825,-157.76768826151715,-105.01100658401518,-44.72958746018019],[-0.024659916758537292,-3.371683120727539,-1.546849250793457,-0.38431036472320557,-0.12505337595939636,-0.1662294864654541],[-0.026901725679636,-3.394101142883301,-1.5401238203048706,-0.388885498046875,-0.12810346484184265,-0.1464039534330368],[-0.6376514909680393,0.2776034348209835,0.7570329763829005,-1.0750878721721475,-1.1302616325684438,0.0453126983717407],[-41.094028857496,11.228906916683105,24.841161461517686,-4.579228925946785,-2.6542899124292636,-3.9239275817378214],[23.18191146850586,24.051803588867188,24.816726684570312,24.939468383789062,24.30146598815918,24.969167709350586]),([513.8879999999999],[-26.880745875806113,57.09885149893252,-157.76837128043414,-44.72614504483851],[0.3945586681365967,-

###  Structural Assessment: Why We Must Define 13 Columns
The reason for defining exactly 13 structural columns is mandatory due to the non-standard format of the raw sensor data and the requirements of the parsing method.

- 1. The Non-Standard Data Problem
     The sensor data files are not standard CSVs. They store all data for a single timestamp on one line, structured as a complex Python tuple:

    (Time, [List of 6 values], [List of 6 values], ..., [List of 6 values]).

If we were to use standard loading methods like pd.read_csv(), Pandas would incorrectly split the data based on all commas (including those inside the lists), resulting in hundreds of meaningless columns (as you previously saw with the 806 column count).

- 2. The Structural Mandate: 13 Top-Level Fields
     To correctly load the data, we must use the ast.literal_eval function. This function safely parses the complex string into a single Python list of its distinct, top-level elements.

When the raw text is parsed, it consistently yields exactly 13 primary fields per row:

---

Total Columns After Expansion

When lists are expanded into joint-indexed columns:

1 scalar column → 1 column

12 list fields × 6 joint values each → 72 columns

Total columns = 1 + 72 = 73

This matches exactly with the 73 header names provided in:

***`ur5testresult_header.xlsx`***

---

In [6]:
import ast

# Define 13 GENERIC column names. This is mandatory to structure the parsed data 
# that results from reading 13 fields per line.
COLUMNS_STRUCTURAL_GENERIC = [f"C{i+1}" for i in range(13)]

all_data = []

print("Starting robust structural parsing and combination...")

# --- Core Parsing Loop ---
# This loop iterates through the 'sensor_files' list generated in the previous step
for fpath in sensor_files:
    with open(fpath, "r") as f:
        for line in f:
            # Strip parentheses and prepare the line for safe evaluation
            line = line.strip().lstrip('(').rstrip(')')
            try:
                # Safely parse the line into a list of 13 elements
                parsed = ast.literal_eval(f"[{line}]")
                all_data.append(parsed)
            except:
                # Skip any malformed lines
                continue

# Create the final structural DataFrame
sensor_data_structural = pd.DataFrame(all_data, columns=COLUMNS_STRUCTURAL_GENERIC)

# --- Inspection of Combined Data ---
print(f"Loaded {len(sensor_files)} sensor files")
print(f"Combined structural dataset shape: {sensor_data_structural.shape}")
print(f"Total Rows: {sensor_data_structural.shape[0]}")
print(f"Total Columns: {sensor_data_structural.shape[1]}")

print("\n--- Structural Data Inspection (First 5 Rows) ---")
display(sensor_data_structural.head())

Starting robust structural parsing and combination...
Loaded 18 sensor files
Combined structural dataset shape: (153658, 13)
Total Rows: 153658
Total Columns: 13

--- Structural Data Inspection (First 5 Rows) ---


,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13
0,[747.2479999999999],"[-26.880068716264294, -79.91160892471794, 57.0...","[-26.876620441547427, -79.91090832767539, 57.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, -0.0, 0.0, 0.0, 0.0]","[-2.2329763972194327e-18, -2.2138144127730115,...","[0.2398737221956253, -3.4344537258148193, -1.8...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-2.6149525002185862e-17, -25.67417061620775, ...","[0.22866468131542206, -3.4344537258148193, -1....","[-0.6377188174812196, 0.2775360293865634, 0.75...","[-25.23140494271829, 17.43970690181103, 6.5165...","[25.209991455078125, 26.71473503112793, 26.712..."
1,[747.256],"[-26.880068716264294, -79.91160892471794, 57.0...","[-26.876620441547427, -79.9102253087584, 57.09...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, -0.0, 0.0, 0.0, 0.0]","[-2.2329763972194327e-18, -2.2138144127730115,...","[0.2398737221956253, -3.4366955757141113, -1.8...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-2.6149525002185862e-17, -25.67417061620775, ...","[0.2376319169998169, -3.4344537258148193, -1.8...","[-0.6377179484974931, 0.27754250895226273, 0.7...","[-27.052612657185563, 17.178791785221726, 8.71...","[25.209991455078125, 26.71473503112793, 26.712..."
2,[747.264],"[-26.880068716264294, -79.91160892471794, 57.0...","[-26.87937983797211, -79.9095422898414, 57.097...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, -0.0, 0.0, 0.0, 0.0]","[-2.2329763972194327e-18, -2.2138144127730115,...","[0.23539011180400848, -3.4322118759155273, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-2.6149525002185862e-17, -25.67417061620775, ...","[0.2376319169998169, -3.4344537258148193, -1.8...","[-0.6377234520452051, 0.2775758667322003, 0.75...","[-26.723421390438553, 16.973602855200063, 8.11...","[25.209991455078125, 26.71473503112793, 26.712..."
3,[747.2719999999999],"[-26.880068716264294, -79.91160892471794, 57.0...","[-26.87937983797211, -79.91090832767539, 57.09...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, -0.0, 0.0, 0.0, 0.0]","[-2.2329763972194327e-18, -2.2138144127730115,...","[0.23314829170703888, -3.4322118759155273, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-2.6149525002185862e-17, -25.67417061620775, ...","[0.2376319169998169, -3.4344537258148193, -1.8...","[-0.6377102675303948, 0.27756214867412954, 0.7...","[-26.701637393048827, 17.399379345542844, 7.90...","[25.209991455078125, 26.71473503112793, 26.712..."
4,[747.28],"[-26.880068716264294, -79.91160892471794, 57.0...","[-26.87730346046443, -79.9095422898414, 57.096...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, -0.0, 0.0, 0.0, 0.0]","[-2.2329763972194327e-18, -2.2138144127730115,...","[0.25780820846557617, -3.4299700260162354, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[-2.6149525002185862e-17, -25.67417061620775, ...","[0.2645336389541626, -3.4344537258148193, -1.8...","[-0.6377319481118858, 0.27754833705585435, 0.7...","[-26.205690541384136, 17.76316899344647, 7.908...","[25.209991455078125, 26.71473503112793, 26.714..."


In [7]:
print("\n--- Data Types and Memory Usage ---")
sensor_data_structural.info()


--- Data Types and Memory Usage ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153658 entries, 0 to 153657
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   C1      153658 non-null  object
 1   C2      153658 non-null  object
 2   C3      153658 non-null  object
 3   C4      153658 non-null  object
 4   C5      153658 non-null  object
 5   C6      153658 non-null  object
 6   C7      153658 non-null  object
 7   C8      153658 non-null  object
 8   C9      153658 non-null  object
 9   C10     153658 non-null  object
 10  C11     153658 non-null  object
 11  C12     153658 non-null  object
 12  C13     153658 non-null  object
dtypes: object(13)
memory usage: 15.2+ MB


###  Cleaning Is Required to Remove Noise and Invalid Characters

- The raw files contain unwanted characters:

    - ( ) tuple wrappers 

    - leading/trailing spaces

    - inconsistent delimiters

    - '[' and ']' list wrappers

    - embedded commas

- These must be handled before expansion, otherwise:

    - Parsing fails 

    - Numeric conversion fails 

    - Merging with header fails 

- Cleaning ensures:

    - All rows are valid Python literals

    - No malformed list structures

    - All values are numeric after expansion

##  Step4 : Summary File Inspection

The summary file `calculated_deviation_of_actual_position_to_nominal_position.xlsx`  
contains deviation statistics between **actual and target joint positions**.  
This serves as a **validation reference** for future analysis and anomaly detection.


In [8]:
# --- Step 4: Summary data ---
summary_df = pd.read_excel(summary_path)
print("Summary file loaded successfully.\n")
display(summary_df.head())

Summary file loaded successfully.



,Calculated deviation of actual position to nominal position,Unnamed: 1
0,Position,delta Distance (mm)
1,position 1,0.156205
2,position 2,0.163095
3,position 3,0.189737
4,position 4,0.17398


In [9]:
print("Sensor Data Columns:", len(sensor_data.columns))
print("Summary Columns:", len(summary_df.columns))

Sensor Data Columns: 806
Summary Columns: 2


### Understanding the Summary File and How to Use It
`calculated_deviation_of_actual_position_to_nominal_position.xlsx`

This summary file contains pre-computed deviation statistics that measure how far the robot’s actual joint positions drifted from the nominal (target) positions during calibration or reference tests.

It serves two important roles:

1. Baseline Reference for System Accuracy

    These deviations represent the robot’s expected baseline performance.

    We can use them to:

    - Evaluate if future sensor data deviates abnormally

    - Compare real-time joint positions against known acceptable offsets

    - Validate whether loaded sensor logs reflect normal operation
  
2. Calibration & Threshold Setting for ML Tasks

    If we later build:

    - Anomaly detection

    - Predictive maintenance models

    - Outlier filtering

    - Drift detection systems

    …this file provides ground-truth thresholds.

## Step 5: Save Structural Data for Next Notebooks

At this stage, the dataset is **structurally aligned but uncleaned**.  
It will be used as input for the next notebook (`02_data_cleaning.ipynb`).

In [10]:
# --- Step 5: Save both CSV and Parquet ---

# Output directory
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

structured_csv = os.path.join(output_dir, "structured_sensor_data.csv")
structured_parquet = os.path.join(output_dir, "structured_sensor_data.parquet")

sensor_data.to_csv(structured_csv, index=False)
sensor_data.to_parquet(structured_parquet, index=False)

print(f"Structured dataset saved at:\n- {structured_csv}\n- {structured_parquet}")


Structured dataset saved at:
- ../data/processed/structured_sensor_data.csv
- ../data/processed/structured_sensor_data.parquet


## Step 6: Assessment & Conclusion: Data Readiness
   This section completes the 01_data_exploration notebook by assessing the structurally parsed data and preparing for the next phase.

   ---
   
   ###  Summary of Findings
   - Structural Integrity: All raw sensor files were successfully parsed and combined into a single, structurally sound DataFrame with 13 columns (rows: $\approx 153,658$).
   - Data Type Status: The data is not yet ML-ready. The majority of the columns are stored as object type (Python lists) and cannot be used directly for modeling.
   - Readiness: The data is ready for the Cleaning and Transformation Phase.
   
   ### Conclusion & Next Step
   The structural DataFrame is the clean input for the next step.
   - Goal Achieved: The fundamental data acquisition and structural validation are complete.
   - Next Phase: The raw list columns must be expanded into $\mathbf{72}$ individual numerical features, and all resulting $\mathbf{73}$ columns must be converted to $\mathbf{\text{float64}}$.
   - File Saved: The 13-column structural dataset has been saved to ../data/processed/sensor_data_structural.parquet.

   ---
   
   **`Next Notebook → 02_data_cleaning.ipynb`**